<a href="https://colab.research.google.com/github/sualeh/introduction-to-chatgpt-api/blob/main/chatgpt-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------

> **How to Run This Notebook**

To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. If you are running the notebook locally in Visual Studio Code or other IDE, create a file called `.env`, and add a line `OPENAI_API_KEY=<your-openai-api-key>`. This key will be read by the `load_dotenv` library.

Otherwise, if you are running in Google Colab, create a secret called `OPENAI_API_KEY` and set it to the value of your OpenAI API key.

Run the code below to read the key.


In [ ]:
%pip install -qq python-dotenv

from os import environ as env
from dotenv import load_dotenv
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Load key from an environmental variable called "OPENAI_API_KEY"
# Use python-dotenv https://pypi.org/project/python-dotenv/
# And take environment variables from .env
load_dotenv()
try:
  # Attempt to read OPENAI_API_KEY from a Google Colab secret
  from google.colab import userdata
  env['OPENAI_API_KEY'] = env.get('OPENAI_API_KEY', userdata.get('OPENAI_API_KEY'))
except ModuleNotFoundError:
  logger.info("Not running in Google Colab")
  # No action - rely on the OPENAI_API_KEY environmental variable



----------

# A Short Introduction to the ChatGPT API


The ChatGPT API is relatively easy to use. It has object bindings in most major languages, so you can use your language of choice.

The API allows you to integrate OpenAI's powerful language models like GPT-4 directly into your applications. Unlike using the ChatGPT web interface, the API gives you programmatic access and more control over the model's behavior, allowing you to build custom AI-powered solutions.

## Getting Started

To use the ChatGPT API, we first need to install the OpenAI Python library and create a client. The client handles authentication and communication with OpenAI's servers, allowing us to make requests to the various models available through the API.

In [ ]:
%pip install -qq openai

from openai import OpenAI

# Create Open AI client to use ChatGPT
client = OpenAI()

## Making a Request

Start writing your first request - you need to specify the model and provide the user prompt.

The "user" role represents the user's input or message within the conversation. The prompt can include questions, commands, or any other content the user wants to communicate.

When making a request to the ChatGPT API, you need to include the following components:

1. **Model**: Choose which LLM model to use (e.g., "gpt-4o")
2. **Messages**: A list of message objects that form the conversation history
3. **Optional parameters**: Such as temperature (controls randomness), max_tokens (limits response length), etc.

Each message in the messages list has a "role" and "content". The role can be "system", "user", or "assistant".

In [ ]:
prompt = "When was the first moon landing?"

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": prompt}
    ],
    model="gpt-4o",
)

print(chat_completion)

The completion has the response content, as well as usage information. Responses are messages from the "assistant" role. What we really want to extract, though, is the actual message content.

In [ ]:
reply = chat_completion.choices[0].message.content
print(reply)

## Using the "system" Role

The "system" role is used to provide high-level instructions or setting the behavior of the assistant. It guides the conversation or specifies the desired outcome. This information is typically provided at the beginning of the conversation.

System messages are powerful because they allow you to shape the assistant's behavior without explicitly including these instructions in every user prompt. Some examples of effective system messages include:

- "You are an expert in physics with a PhD. Explain concepts in simple terms."
- "Respond with just a single word or short phrase."
- "Always include code examples in Python when explaining programming concepts."
- "You are a helpful assistant that speaks like Shakespeare."

System messages help establish the "personality" and behavior patterns of the AI assistant throughout the conversation.

In [ ]:
prompt = "When was the first moon landing?"

chat_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Always reply in French."},
        {"role": "user", "content": prompt},
    ],
)

reply = chat_completion.choices[0].message.content
print(reply)

ChatGPT is most useful when it has context from the conversation. You will need to maintain chat history yourself, by keeping the "system" and "user" messages as well as the "assistant" reponses in a list, and passing this list into every API call.

## Maintaining Context

ChatGPT is a REST API, and does not maintain sessions. However, the client can maintain its own chat history, and present that on an API call for additional context on the conversation. To be effective, you would need to maintain both the user questions and the ChatGPT responses. In its simplest form, it is a list of chat messages. By maintaining and passing the conversation history in each request, you can create the illusion of a continuous conversation with the AI, even though each request is technically separate.

In [ ]:
prompt = "What is the moon's circumference in km?"

message_history = []
message_history.append(
    {"role": "user", "content": prompt}
)

chat_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=message_history
)

# Print the response:
reply_content = chat_completion.choices[0].message.content
print(reply_content)

Let us store the response in the history as well, so we can make a context-sensitive prompt.

In [ ]:
message_history.append({"role": "assistant", "content": f"{reply_content}"})

Now, ask an ambiguous question, which is answered in context.

Notice how we're asking "How far is it?" - without context, this question is meaningless. However, since we've included the previous messages in our history, the AI understands we're referring to the moon's circumference mentioned in the previous exchange.

This demonstrates why maintaining conversation history is crucial for creating natural, flowing interactions with AI assistants. Without this context, you would need to restate your entire question with all relevant details each time.

In [ ]:
prompt = "How far is it?"

message_history.append({"role": "user", "content": prompt})

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=message_history
)

reply_content = chat_completion.choices[0].message.content
print(reply_content)

## References

- Based on [Sentdex/ChatGPT-API-Basics](https://github.com/Sentdex/ChatGPT-API-Basics)
- See [ChatGPT API in Python by sentdex](https://www.youtube.com/watch?v=c-g6epk3fFE)